<h2> Stock Market Prediction Using LSTM Networks </h2>

In [ ]:
# Importing the necessary libraries.
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from statsmodels.tsa.arima.model import ARIMA

# Import the gold.csv file
df = pd.read_csv(r'Project Code/tesla.csv')

# Display the first 5 rows of the dataframe
df.head()

In [ ]:
df.describe()

<p> The standard deviation (std) in the `describe()` function quantifies the variation or dispersion within the data set. A higher standard deviation signifies that the 

data points are dispersed over a broader range of values. In contrast, a lower standard deviation indicates that the data points cluster closely around the mean. </p>


<p> Standard Deviation for Key Columns

- Open: The standard deviation is 110.838840, indicating that the opening prices exhibit considerable variability from the mean opening price of 84.068473.
- High: The standard deviation is 113.321844, demonstrating that the highest prices throughout the day also show significant fluctuations around the mean of 85.916742.
- Low: The standard deviation is 108.110895, suggesting that the lowest prices during the day are similarly spread around the mean of 82.091640.
- Close: The standard deviation is 110.747440, indicating that the closing prices vary significantly around the mean closing price of 84.042719.

Other Columns:
- Volume: The standard deviation is 7.744906e+07, reflecting a substantial variation in trading volume compared to the mean of 9.651920e+07.
- Dividends: The standard deviation is 0.0, which signifies no variation in dividend values, as all entries are uniformly 0.
- Stock Splits: The standard deviation is 0.096083, indicating minimal variation around the mean of 0.002173.</p>

<h3> Data Cleaning </h3>

In [ ]:
df.reset_index(drop = True, inplace = True) # Resetting the index.
df.sort_values(by = 'Date', ascending = True, inplace = True) # Sorting the data by 'Date' in ascending order.

df['Date'] = pd.to_datetime(df['Date'], errors = 'coerce', utc = True) # Converting the 'Date' column to datetime.
df.info()

<h3> Visualisation of the data </h3>

In [ ]:
# Visualise the closing price of the stock.
plt.figure(figsize = (14, 7))
plt.plot(df['Date'], df['Close'], label = 'Close Price')
plt.title('Tesla Stock Closing Price') 
plt.xlabel('The Date') 
plt.ylabel('The Closing Price')
plt.legend()
plt.show() 

# visualise the volume of stocks sold each year
plt.figure(figsize = (15, 6))
df.set_index('Date')['Volume'].plot()
plt.xlabel('Year')
plt.ylabel('Volume')
plt.title("Sales Volume of Tesla")
plt.tight_layout()
plt.show()

In [ ]:
# Extract year and calculate average closing price per year
df['Year'] = df['Date'].dt.year
yearly_avg = df.groupby('Year')['Close'].mean()

# Plot the data
plt.figure(figsize = (10,5))
plt.plot(yearly_avg.index, yearly_avg.values, marker = 'o', linestyle = '-', color = 'b', label = 'Average Yearly Close')
plt.xlabel('Year')
plt.ylabel('Average Closing Price')
plt.title('Yearly Average Closing Prices')
plt.legend()
plt.grid()
plt.show()

# Daily stock returns.
df['Daily Return'] = df['Close'].pct_change()
plt.figure(figsize= ( 12,6))
plt.plot(df['Date'], df['Daily Return'], color = "red", label = "Daily Returns")
plt.axhline(0, color = "black", linestyle = "--")
plt.xlabel("Date")
plt.ylabel("Daily Return")
plt.title("Daily Stock Returns Over Time")
plt.legend()
plt.show()

In [ ]:
print(df['Daily Return'].std()) # Standard deviation of the daily returns.

<p>

The output 0.03614 represents the standard deviation of daily returns in the dataset. A value of around 0.036 indicates the volatility of these returns, with higher values signifying more significant fluctuations and lower values suggesting more stability. This metric is crucial for assessing risk, as it measures how much returns deviate from the mean, helping investors gauge the asset's potential unpredictability.

</p>

In [ ]:
# Calculate the moving averages.
df['50-day MA'] = df['Close'].rolling(window = 50).mean()
df['200-day MA'] = df['Close'].rolling(window = 200).mean()
plt.figure(figsize = (12,6))
plt.plot(df['Date'], df['Close'], label = "Closing Price", alpha = 0.5)
plt.plot(df['Date'], df['50-day MA'], label = "50-day MA", color = "red")
plt.plot(df['Date'], df['200-day MA'], label = "200-day MA", color = "green")
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("Stock Closing Price with Moving Averages")
plt.legend()
plt.show()

# Drop non-numeric columns.
df_numeric = df.select_dtypes(include = ['number'])

# Calculate the correlation matrix.
corr_matrix = df_numeric.corr()

# Plot the correlation heatmap.
plt.figure(figsize = (8,6))
sns.heatmap(corr_matrix, annot = True, cmap = 'coolwarm', fmt = ".2f")
plt.title("Correlation Heatmap")
plt.show()

<p> Price-related variables, namely Open, High, Low, and Close, exhibit a high correlation (approximately ~1.00). This is understandable, as these values typically move in tandem with stock prices.

The 50-day moving average (MA) and 200-day MA show a strong correlation (around ~0.96-0.98) with stock prices. Moving averages serve to smooth price trends and inherently follow price movements. In contrast, trading volume demonstrates only a weak correlation (ranging from ~0.08 to 0.35) with price changes, indicating that volume does not significantly impact price fluctuations. Daily returns exhibit minimal correlation with other features (approximately ~-0.02 to 0.05), suggesting that stock returns fluctuate relatively independently of individual price points. 

Finally, the year variable reflects a moderate correlation (around ~0.80) with stock prices, which may indicate an upward trend over time.</p>

In [ ]:
# Sentiment Analysis.
df['Date'] = pd.to_datetime(df['Date'], errors = 'coerce', utc = True)  # Force UTC.
df['Month'] = df['Date'].dt.month 
df['Month'] = df['Date'].dt.month
monthly_avg = df.groupby('Month')['Close'].mean()
monthly_avg.plot(kind = 'bar', title = "Monthly Average Closing Price", figsize = (10,5))

<p>
The graph depicts the monthly average closing price, starting and finishing at 100, with significant fluctuations observed throughout the year. A downward trend begins in month two, gradually declining until it drops sharply in month five, representing the period's lowest point. This mid-year decline suggests a temporary market correction or the influence of external economic factors. Nevertheless, the recovery by the year's end demonstrates resilience and a return to stability. Overall, the observed pattern illustrates cyclical volatility, wherein prices are influenced by seasonal trends, market events, or investor behaviour, ultimately regaining strength by year-end.
</p>

<h3> Building the LSTM Model </h3>

<h4> Train The Model on Close Price 80 - 20 Spilt   </h4>

In [ ]:
df.head()

In [ ]:
# Convert Date column to datetime and sort the data
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by = 'Date')

# Select only the 'Close' price
close_model_df = df[['Date', 'Close']]

# Normalize the 'Close' column
scaler = MinMaxScaler(feature_range = (0,1))
close_model_df['Close'] = scaler.fit_transform(close_model_df[['Close']])

# Display first few rows
close_model_df.head()

In [ ]:
# Function to create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])  # Last `seq_length` days
        y.append(data[i + seq_length])   # Predict next day
    return np.array(X), np.array(y)

# Convert Close column to NumPy array
close_prices = close_model_df['Close'].values

# Define sequence length (e.g., 50 days)
seq_length = 50

# Create sequences
X, y = create_sequences(close_prices, seq_length)

# Split into training (80%) and testing (20%) sets
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# Reshape for LSTM (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Print shapes for verification
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

<p>

The shapes of X_train and X_test are adjusted to meet the input requirements of the LSTM model, which expects a 3D input structure comprising samples, time steps, and features (e.g. the closing price). Specifically:

- Samples denote the number of sequences.
- Time steps represent the length of each sequence (e.g., 50 days).
- Features indicate the number of variables at each time step (1 for univariate time series).

This data restructuring ensures that the LSTM can effectively learn temporal dependencies.

</p>

<p>

Layer Type | Purpose

LSTM (50 units, return_sequences = True): This initial LSTM layer is designed to learn patterns from the past 50 days. We use `return_sequences = True` because another LSTM layer follows it.

Dropout (0.2): This dropout layer randomly removes 20% of neurons to help prevent overfitting.

LSTM (50 units, return_sequences = False): The second and final LSTM layer extracts meaningful features from the data. We set `return_sequences = False` since no additional LSTM layers follow.

Dropout (0.2): Another dropout layer is included to mitigate overfitting.

Dense (25 units, activation = 'relu'): This fully connected layer consists of 25 neurons, and it aims to transform the features derived from the LSTM layers into a more meaningful representation.

Dense (1 unit): This is the output layer, which predicts the stock price for the next day.

</p>

In [ ]:
# Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences = True, input_shape = (50, 1)),  # LSTM layer with 50 units
    Dropout(0.2),  # Dropout to prevent overfitting
    LSTM(50, return_sequences = False),  # Another LSTM layer
    Dropout(0.2),
    Dense(25, activation = 'relu'),  # Dense layer with 25 neurons
    Dense(1)  # Output layer (predict next close price)
])

# Compile the model
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

# Print model summary
model.summary()

<p>

This output summarises the architecture for an LSTM-based sequential model created using Keras. The model features two LSTM layers, each containing 50 units, succeeded by dropout layers with a 20% dropout rate to mitigate overfitting. Following the LSTM layers, there are two dense (fully connected) layers: one with 25 neurons utilising ReLU activation and a final output layer with a single neuron designed to predict the next closing price.

The parameter counts reflect the number of trainable weights within each layer:
- The first LSTM layer has 10,400 parameters.
- The second LSTM layer contains 20,200 parameters.
- The dense layers have 1,275 and 26 parameters, respectively.

Overall, the model encompasses 31,901 trainable parameters. The loss function employed is a mean squared error (MSE), and optimisation is carried out using the Adam optimiser. Mean absolute error (MAE) is tracked as a metric.

The Adam optimiser, or Adaptive Moment Estimation, is preferred for training LSTM models due to its blend of efficiency, stability, and adaptability, which are essential for handling the complexities of LSTM networks. By combining the advantages of Momentum and RMSProp, Adam maintains moving averages of gradients and squared gradients, allowing it to adjust learning rates for each parameter individually. This leads to faster and more reliable convergence, particularly in models with complex error surfaces.

</p>

In [ ]:
# Define early stopping to stop if validation loss doesn't improve for 5 epochs
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    epochs = 50,  # We can set a high number, and early stopping will stop it earlier
    batch_size = 32,
    validation_data = (X_test, y_test),
    callbacks = [early_stopping],
    verbose=1
)

<p>

This output illustrates the training process of an LSTM model designed to predict stock closing prices. The model was trained for 50 epochs with an early stopping mechanism. This mechanism monitors the validation loss (val_loss) and halts training if there is no improvement over five consecutive epochs while restoring the best weights identified during training.

Performance Details:
- Loss (loss) refers to the model's error on the training dataset, which is measured using Mean Squared Error (MSE).
- Validation Loss (val_loss) indicates the model's error on the unseen validation dataset using MSE. Lower values indicate better generalisation.
- Mean Absolute Error (MAE), represented as mae and val_mae, reveals the average absolute difference between predicted and actual values, providing an intuitive understanding of prediction accuracy.

Observations:
In the early epochs (Epoch 1), the training loss begins at 0.0130, while the validation loss is at 0.0031, suggesting that the model is actively learning. As training advances, the loss values decrease noticeably. For instance, by Epoch 10, the training loss has diminished to approximately 5.01e-04, and the validation loss has reduced to 0.0014. 

In the later epochs, both training and validation losses stabilise. By Epochs 23-24, the training loss hovers around ~3.6e-04, stabilising the validation loss between ~0.0010 and 0.0012. This indicates that the model has converged and is neither overfitting nor underfitting.

The LSTM model exhibits impressive performance, as indicated by its low training and validation losses, suggesting that it has successfully captured the underlying patterns of stock closing prices. The low validation MAE values (falling below 0.03) further indicate that the model produces relatively accurate predictions on the validation dataset. The incorporation of early stopping has effectively mitigated overfitting by halting training once the validation performance reached a plateau.

</p>

In [ ]:
# Plot loss curves
plt.figure(figsize = (10, 5))
plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Training vs. Validation Loss')
plt.legend()
plt.show()

<p>

The Training vs. Validation Loss graph shows how the model's Mean Squared Error (MSE) changes over epochs for both datasets. The blue line for Training Loss decreases smoothly and remains low, indicating a good fit to the training data. In contrast, the orange Validation Loss line fluctuates more, suggesting variability in generalisation to unseen data but generally trends downward—a positive sign. The small gap between training and validation losses indicates minimal overfitting. However, spikes in validation loss may suggest sensitivity to specific data points or slight overfitting in later epochs.

</p>

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Reverse the normalization (convert back to actual price)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_actual = scaler.inverse_transform(y_pred)

# Plot actual vs predicted prices
plt.figure(figsize = (10, 5))
plt.plot(y_test_actual, label = 'Actual Prices', color = 'blue')
plt.plot(y_pred_actual, label = 'Predicted Prices', color = 'red')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.title('Actual vs. Predicted Stock Prices')
plt.legend()
plt.show()

<p>

The predicted prices generated by the LSTM model (represented by the red line) closely align with the actual stock prices (illustrated by the blue line) over the entire period. These predictions effectively capture the overarching trend and the short-term fluctuations of the actual prices, showcasing the model's capability to learn intricate patterns within the data. The minimal difference between the predicted and actual prices reflects the high accuracy and reliability of the LSTM model in forecasting stock prices in this instance.

</p>

In [ ]:
# Evaluate on the test set.
mse, mae = model.evaluate(X_test, y_test, verbose = 1)
print(f"Test MSE: {mse:.5f}")
print(f"Test MAE: {mae:.5f}")

# Gives the error in the model.
rmse = np.sqrt(mse)
print(f'RMSE: {rmse:.2f}')

# How well does the model explain the variance.
r2 = r2_score(y_test_actual, y_pred_actual)
print(f'R² Score: {r2:.4f}')

# Calculate the model accuracy.
accuracy_percentage = r2 * 100
print(f'Model Accuracy: {accuracy_percentage:.2f}%')

<p>

- Test MSE: 0.00155 — This value represents the Mean Squared Error on the test dataset, demonstrating a very low average squared difference between the predicted and actual closing prices, which indicates high prediction accuracy.  
- Test MAE: 0.02975 — The Mean Absolute Error for the test data reveals that the model's average predictions deviate by approximately 0.02314 units (likely aligned with the scale used for the normalised or scaled stock prices), highlighting the model's strong performance.  

- Root Mean Squared Error (RMSE): 0.04 - This indicates that, on average, predicted stock prices deviate from actual prices by only 0.03 units, reflecting high accuracy in the model's predictions.

- R² Score (Coefficient of Determination): 0.9194 - This score signifies that the LSTM model explains 94.7% of the variance in stock prices, demonstrating an excellent fit and the ability to identify patterns from historical data.

- Model Accuracy: 91.94% - Regression models typically do not use accuracy percentages, but this score highlights the model's reliability in explaining variance.

In summary, the low RMSE, high R² score, and substantial accuracy collectively emphasize the LSTM network's effectiveness in forecasting stock prices. The model shows strong performance on training and test datasets, with a validation curve that aligns with the training curve and a minimal Mean Absolute Error (MAE), indicating accurate predictions of unseen stock price data.

</p>

<h4> Train The Model on Close & Volume 80 - 20 Spilt </h4>

In [ ]:
# Select only the 'Close' price
close_volume_model_df = df[['Date', 'Close', 'Volume']]

# Normalize 'Close' and 'Volume' separately
scaler = MinMaxScaler(feature_range=(0,1))
close_volume_model_df[['Close', 'Volume']] = scaler.fit_transform(close_volume_model_df[['Close', 'Volume']])

# Display first few rows
close_volume_model_df.head()

In [ ]:
# Function to create sequences with multiple features
def create_sequences_multifeature(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])  # Use both 'Close' and 'Volume'
        y.append(data[i + seq_length, 0])  # Predict only 'Close' price
    return np.array(X), np.array(y)

# Convert dataframe to NumPy array
data_values = close_volume_model_df[['Close', 'Volume']].values

# Define sequence length (e.g., 50 days)
seq_length = 50

# Create sequences
X, y = create_sequences_multifeature(data_values, seq_length)

# Split into training (80%) and testing (20%) sets
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# Reshape for LSTM (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 2))  # 2 features
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 2))

# Print shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

<p>

The shapes of X_train and X_test are reshaped to (samples, time steps, and features) to meet the input requirements of the LSTM model for multivariate time series data. In this context, two features (such as 'Close' price and 'Volume') are utilised, resulting in a shape that represents:

- Samples: The number of sequences,
- Time steps: The length of each sequence (for instance, 50 days),
- Features: The number of variables at each time step (2).

This configuration enables the LSTM to learn from multiple features concurrently, thereby capturing more nuanced temporal patterns to enhance prediction accuracy.

</p>

In [ ]:
# Build the LSTM model with multiple features
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 2)),  # 2 features now
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)  # Predicting 'Close' price only
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print model summary
model.summary()

<p>

The model architecture consists of LSTM layers. The first LSTM layer contains 50 units and is configured with `return_sequences = True`. It accepts an input shape of `(seq_length, 2)`. The second LSTM layer has 50 units but is set to `return_sequences = False`, producing a single output vector. Two dropout layers with a 20% dropout rate are added after each LSTM layer to reduce overfitting.

Following the LSTM layers, there is a dense layer with 25 units and ReLU activation, which feeds into a final layer with a single unit to predict the 'Close' price. The model utilises the Adam Optimiser and employs Mean Squared Error (MSE) as the loss function, with Mean Absolute Error (MAE) as a metric for accuracy. The first LSTM layer has 10,600 parameters, the second has 20,200, and the dense layers have 1,275 and 26 parameters, resulting in 32,101 trainable parameters. This model aims to capture temporal dependencies in the data to improve stock price predictions. The dropout layers help promote generalisation, while the dense layers enhance accuracy. 

Overall, the architecture is designed for effective time-series forecasting by balancing complexity and performance.

</p>

In [ ]:
# Define early stopping to stop if validation loss doesn't improve for 5 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience = 5, restore_best_weights = True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs = 50,
    batch_size = 32,
    validation_data = (X_test, y_test),
    callbacks = [early_stopping],
    verbose = 1
)

In [ ]:
# Plot training vs. validation loss.
plt.figure(figsize = (10, 5))
plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Training vs. Validation Loss')
plt.legend()
plt.show()

<p>

The "Training vs. Validation Loss" graph illustrates the LSTM model's performance regarding stock closing prices and volume across training epochs. The blue line represents Training Loss, while the orange line denotes Validation Loss. Both metrics show a significant decrease early on, indicating effective learning. The convergence of the two lines suggests good generalization without overfitting, and the stable validation loss of around 0.002 demonstrates the model's substantial predictive accuracy for stock prices, confirming the LSTM's ability to capture underlying data patterns.


</p>

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Reverse normalization
y_test_actual = scaler.inverse_transform(np.column_stack((y_test, np.zeros_like(y_test))))[:, 0]
y_pred_actual = scaler.inverse_transform(np.column_stack((y_pred.flatten(), np.zeros_like(y_pred.flatten()))))[:, 0]

# Plot actual vs. predicted prices
plt.figure(figsize = (10, 5))
plt.plot(y_test_actual, label = 'Actual Prices', color = 'blue')
plt.plot(y_pred_actual, label = 'Predicted Prices', color = 'red')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.title('Actual vs. Predicted Stock Prices')
plt.legend()
plt.show()

<p>

The graph titled "Actual vs Predicted Stock Prices" illustrates a comparison between the predictions made by the LSTM model (represented by the red line) and the actual stock prices (shown by the blue line). The predicted prices closely align with the actual values, showcasing the model's impressive ability to capture trends and patterns within the data. Although there are minor discrepancies, the overall correspondence reflects a high level of accuracy in forecasting. This performance underscores the effectiveness of the LSTM model in utilising historical price and volume data for reliable predictions, establishing it as a valuable asset for stock price analysis.

</p>

In [ ]:
# Evaluate model.
mse, mae = model.evaluate(X_test, y_test, verbose = 1)
print(f"Test MSE: {mse:.5f}")
print(f"Test MAE: {mae:.5f}")

# Gives the error in the model.
rmse = np.sqrt(mse)
print(f'RMSE: {rmse:.2f}')

# How well does the model explains the variance.
r2 = r2_score(y_test_actual, y_pred_actual)
print(f'R² Score: {r2:.4f}')

# Calculate the model accuracy.
accuracy_percentage = r2 * 100
print(f'Model Accuracy: {accuracy_percentage:.2f}%')

<p>

The results display the LSTM model's performance on the test set, revealing a Test MSE of 0.00105 and a Test MAE of 0.02379. The low MSE reflects minimal squared prediction errors, while the low MAE indicates that the model's predictions closely align with the actual stock prices on average. These findings underscore the model's high accuracy and reliability in forecasting stock prices.

Root Mean Squared Error (RMSE): 0.03 - This indicates that, on average, predicted stock prices deviate from actual prices by only 0.03 units, reflecting high accuracy in the model's predictions. R² Score (Coefficient of Determination): 0.9453 - This score signifies that the LSTM model explains 94.7% of the variance in stock prices, demonstrating an excellent fit and the ability to identify patterns from historical data. Model Accuracy: 94.53% - Regression models typically do not use accuracy percentages, but this score highlights the model's reliability in explaining variance.

</p>

<h4> Comparison against ARIMA Model </h4>

In [ ]:
# Convert the 'Close' column to a numpy array.
close_prices = df['Close'].values

# Define training data length (80% of the data).
training_data_len = int(len(close_prices) * 0.8)

# Split the data into training and testing sets.
train_data = close_prices[:training_data_len]
test_data = close_prices[training_data_len:]

# Fit the ARIMA model on the training data.
arima_model = ARIMA(train_data, order = (5, 1, 0))
arima_model_fit = arima_model.fit()

# Make predictions for the test set period.
start_index = training_data_len
end_index = len(close_prices) - 1

arima_predictions = arima_model_fit.predict(start = start_index, end = end_index, typ = 'levels')

# Align predictions to match y_test length (if necessary).
aligned_arima_predictions = arima_predictions[:len(y_test)]

# Evaluate the ARIMA model using MSE and MAE for consistency.
mse = np.mean((aligned_arima_predictions - y_test) ** 2)
mae = np.mean(np.abs(aligned_arima_predictions - y_test))

# Print the MSE and MAE.
print(f"Test MSE: {mse:.5f}")
print(f"Test MAE: {mae:.5f}")

<p>

The ARIMA model shows a high Test MSE of 81273.41844 and a Test MAE of 285.08490, indicating poor predictive performance compared to the LSTM model, which has much lower errors with a Test MSE of 0.00074 and a Test MAE of 0.02051. ARIMA's linear design limits its ability to capture complex patterns and non-linear relationships in the data. In contrast, the LSTM model excels in accuracy and robustness by effectively learning sequential dependencies and non-linear trends. This comparison highlights the limitations of traditional models like ARIMA and the advantages of advanced approaches like LSTMs in predictive analytics.

</p>

In [ ]:
# Make predictions with ARIMA.
arima_predictions = arima_model_fit.predict(start = training_data_len, end = len(close_prices) - 1, typ = 'levels')

# Align the lengths of arima_predictions and y_test.
aligned_arima_predictions = arima_predictions[:len(y_test)]

# Reverse normalisation for y_test (actual prices).
# Assuming y_test is a 1D array and scaler was fit on the entire dataset.
y_test_reshaped = y_test.reshape(-1, 1)
y_test_actual = scaler.inverse_transform(np.hstack((y_test_reshaped, np.zeros_like(y_test_reshaped))))[:, 0]

# Reverse normalisation for ARIMA predictions.
arima_predictions_reshaped = aligned_arima_predictions.reshape(-1, 1)
arima_predictions_actual = scaler.inverse_transform(np.hstack((arima_predictions_reshaped, np.zeros_like(arima_predictions_reshaped))))[:, 0]

# Plot the actual vs. ARIMA predicted prices.
plt.figure(figsize = (10, 5))
plt.plot(y_test_actual, label = 'Actual Prices', color = 'blue')
plt.plot(arima_predictions_actual, label = 'ARIMA Predictions', color = 'red')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.title('Actual vs. ARIMA Predicted Stock Prices')
plt.legend()
plt.show()

<p>

The analysis shows a significant gap between actual stock prices and those predicted by the ARIMA model, which struggles due to its linearity. ARIMA typically forecasts trends based on historical data but fails to capture sudden shifts or periods of stability, resulting in unrealistic constant predictions. While actual prices fluctuate realistically over time, ARIMA predictions remain flat at around 138,000, indicating an inability to recognize underlying patterns. This limitation stems from the model's assumptions of linearity and stationarity, making it unsuitable for the non-linear, volatile nature of financial time series. A more effective approach would be to use LSTM models or other machine learning techniques for better forecasting.

Comparison against the LSTM Model predictions

The predicted stock prices from the LSTM model closely align with the actual prices, showcasing the model's proficiency in capturing complex trends and fluctuations in the data. In contrast, the predictions from the ARIMA model significantly diverge from the actual values, with forecasts remaining unrealistically elevated and disconnected from reality. This stark contrast underlines the superior performance of the LSTM model over the ARIMA model in this instance, making it a more suitable choice for the dataset.

</p>

In [ ]:
# Print ARIMA model summary.
print(arima_model_fit.summary())

<p>

1. Model Overview
   - ARIMA(5, 1, 0): This model features five autoregressive (AR) terms, one order of differencing (I), and no moving average (MA) terms.
   - Log Likelihood: -7999.619, reflecting the model’s fit to the data (higher values signify better fit).
   - AIC/BIC/HQIC: These are information criteria used for model comparison, where lower values indicate a preferable model. The AIC is 16011.238, BIC is 16047.163, and HQIC is 16024.173.

2. Coefficients
   - AR Terms: The coefficients for the AR terms (ar.L1 to ar.L5) illustrate their influence on the model. For instance, ar.L1 has a coefficient of -0.0274, suggesting a negative relationship with its prior value.
   - Significance: P-values (P>|z|) denote the significance of each term. Terms with p-values < 0.05, such as ar.L1, ar.L2, ar.L3, and ar.L5, are statistically significant.

3. Residuals
   - Sigma2: The variance of the residuals is calculated to be 13.4186.
   - Ljung-Box Test: A p-value of 0.97 indicates no significant autocorrelation in the residuals, which is favourable.
   - Jarque-Bera Test: A p-value of 0.00 suggests that the residuals do not follow a normal distribution, which could be a concern.
   - Heteroskedasticity: A p-value of 0.00 indicates the presence of heteroskedasticity, suggesting that the variance of residuals is not constant.

4. Diagnostics
   - Skewness and Kurtosis: The residuals are markedly non-normal with a skewness of 0.42 and a kurtosis of 45.65.

5. Warnings
   - The covariance matrix is estimated using the outer product of gradients, which is a standard method but may not be the most optimal in every situation.

In summary, the ARIMA(5, 1, 0) model effectively captures some significant autoregressive patterns; however, it exhibits residual normality and heteroskedasticity issues that could impact its predictive capabilities. Further refinement or exploration of alternative models may be advantageous.

</p>

In [ ]:
# In-sample predictions (on training data).
arima_in_sample_predictions = arima_model_fit.predict(start = 0, end = training_data_len - 1, typ = 'levels')

# Calculate errors (Mean Squared Error).
in_sample_mse = np.mean((arima_in_sample_predictions - close_prices[:training_data_len]) ** 2)
out_sample_mse = np.mean((aligned_arima_predictions - y_test_actual) ** 2)

# Create lists for plotting.
epochs = [1, 2]  # Only two points: In-sample (train) and Out-of-sample (val).
training_mse_values = [in_sample_mse, in_sample_mse]  # Training MSE (constant for visualisation).
validation_mse_values = [in_sample_mse, out_sample_mse]  # Validation MSE (starts at training MSE, ends at validation MSE).

# Plot "Training vs. Validation Loss" for ARIMA.
plt.figure(figsize = (8, 5))
plt.plot(epochs, training_mse_values, label = 'Training MSE', marker = 'o', linestyle = '-', color = 'blue')
plt.plot(epochs, validation_mse_values, label = 'Validation MSE', marker = 'o', linestyle = '-', color = 'red')
plt.xticks([1, 2], ['Epoch 1', 'Epoch 2'])  # Label epochs for clarity.
plt.ylabel('MSE')
plt.title('ARIMA Training vs. Validation Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

<p>

The graph titled "ARIMA Training vs. Validation Loss (MSE)" illustrates the Mean Squared Error (MSE) for both the training and validation datasets over two epochs. An epoch is a complete pass through the entire training dataset in a machine learning model's training. It signifies one complete cycle during which the model has had the opportunity to observe and learn from all the training data at least once. The number of epochs is a hyperparameter that can impact the training duration and the model's performance.

The Training MSE, represented by the blue line, remains stable, indicating the model's consistent performance on the training data. The Validation MSE, shown by the red line, begins at the same level as the Training MSE in Epoch 1 but decreases significantly in Epoch 2, suggesting improved generalization to unseen data. However, the widening gap between Training and Validation MSE may indicate potential overfitting, significantly if Validation MSE rises in future epochs. Ongoing monitoring is essential to maintain performance.

</p>